# Open Clusters Analysis - 01_Downloading data

In [1]:
# Import neccessary libraries

import numpy as np
import pandas as pd
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u
import getpass
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported")


Libraries imported


In [2]:
cluster_name='NGC 2567'

custom_simbad = Simbad()

# Reset SIMBAD to basic configuration
Simbad.reset_votable_fields()

# Add specific fields
custom_simbad.add_votable_fields('otype', 'dim', 'plx', 'pmra', 'pmdec','dim')

result = custom_simbad.query_object(cluster_name)

print(f"Number of results: {len(result)}")

Number of results: 1


In [3]:
# Show basic information from SIMBAD
row = result[0]
ra = row['ra']
dec = row['dec']
radius = row['galdim_majaxis']/2

print(f"\nBasic information:")
print(f"  Name: {row['main_id']}")
print(f"  Type: {row['otype']}")
print(f"  RA: {row['ra']:.6f}º")
print(f"  Dec: {row['dec']:.6f}º")
print(f"  PmRA: {row['pmra']} (mas/yr)") # proper motion ra mas/yr
print(f"  PmDec: {row['pmdec']} (mas/yr)") # proper motion dec mas/yr
print(f"  galdim_majaxis: {row['galdim_majaxis']} (arcmin)") #major axis arcmin
print(f"  galdim_minaxis: {row['galdim_minaxis']} (arcmin)") #minor axis arcmin
print(f"  Radius: {row['galdim_majaxis']/2} (arcmin)")
print(f"  Parallax: {row['plx_value']} (mas)")       


Basic information:
  Name: NGC  2567
  Type: OpC
  RA: 124.653900º
  Dec: -30.637500º
  PmRA: -2.955 (mas/yr)
  PmDec: 2.58 (mas/yr)
  galdim_majaxis: 7.699999809265137 (arcmin)
  galdim_minaxis: 7.699999809265137 (arcmin)
  Radius: 3.8499999046325684 (arcmin)
  Parallax: 0.542 (mas)


In [ ]:
# Request credentials to connect to Gaia
authenticated = False
username = "user"
password = "pass"

try:
    Gaia.login(user=username, password=password)
    print("Login successful")
    authenticated = True
    
except Exception as e:
    print(f"Login error: {e}")
    print("continue without login")


INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
Login successful


In [5]:
max_sources=5000

# Consulta ADQL para obtener datos básicos de astrometría y fotometría
query = f"""
SELECT TOP {max_sources}
    source_id,
    ra, dec,
    parallax, parallax_error,
    pmra, pmra_error, 
    pmdec, pmdec_error,
    phot_g_mean_mag,
    phot_bp_mean_mag, 
    phot_rp_mean_mag,
    bp_rp,
    ruwe,
    visibility_periods_used
FROM gaiadr3.gaia_source 
WHERE CONTAINS(POINT('ICRS', ra, dec), 
                CIRCLE('ICRS', {ra}, {dec}, {radius/60.0})) = 1
AND parallax IS NOT NULL
AND parallax > -5
AND phot_g_mean_mag IS NOT NULL
AND phot_g_mean_mag < 20
ORDER BY phot_g_mean_mag ASC
"""

In [6]:
try:
    if authenticated:
        # Launch asynchronous query (better for big query)
        print('Launching async job')
        job = Gaia.launch_job_async(query)
        results = job.get_results()
    else:
        # Launch synchronous query (anonymous)
        print('Launching sync job')        
        job = Gaia.launch_job(query)
        results = job.get_results()
        
    print(f"Number of results: {len(results)}")
    
except Exception as e:
    print(f"Error executing the query in Gaia: {e}")


Launching async job
INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 929


In [7]:
df = results.to_pandas()
df.head()

,source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,ruwe,visibility_periods_used
0,5644230997165132928,124.646915,-30.649373,0.356224,0.049704,-3.373785,0.041812,1.540275,0.048808,10.821876,11.260475,10.118810,1.141665,4.684865,28
1,5644231070190281088,124.610143,-30.658426,0.593840,0.010241,-2.877418,0.008561,2.713410,0.009848,10.842168,11.392066,10.140247,1.251819,0.902825,28
2,5644229592721548672,124.592797,-30.662180,0.592197,0.014534,-2.958140,0.012551,2.732357,0.013663,11.023625,11.252785,10.640217,0.612568,1.058135,27
3,5644231482507033088,124.695883,-30.612899,0.556131,0.012453,-2.682053,0.010377,2.340598,0.011188,11.068999,11.600001,10.384116,1.215885,0.958634,28
4,5644231104550010496,124.612818,-30.646670,0.493254,0.046313,-2.609204,0.039167,2.799766,0.044128,11.133091,11.286654,10.855161,0.431494,3.088783,28


In [8]:
df.head(15)

,source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,ruwe,visibility_periods_used
0,5644230997165132928,124.646915,-30.649373,0.356224,0.049704,-3.373785,0.041812,1.540275,0.048808,10.821876,11.260475,10.118810,1.141665,4.684865,28
1,5644231070190281088,124.610143,-30.658426,0.593840,0.010241,-2.877418,0.008561,2.713410,0.009848,10.842168,11.392066,10.140247,1.251819,0.902825,28
2,5644229592721548672,124.592797,-30.662180,0.592197,0.014534,-2.958140,0.012551,2.732357,0.013663,11.023625,11.252785,10.640217,0.612568,1.058135,27
3,5644231482507033088,124.695883,-30.612899,0.556131,0.012453,-2.682053,0.010377,2.340598,0.011188,11.068999,11.600001,10.384116,1.215885,0.958634,28
4,5644231104550010496,124.612818,-30.646670,0.493254,0.046313,-2.609204,0.039167,2.799766,0.044128,11.133091,11.286654,10.855161,0.431494,3.088783,28
5,5644231585586232960,124.700063,-30.592808,0.578067,0.019742,-2.956552,0.017360,2.667442,0.017769,11.192613,11.240378,11.090806,0.149572,1.165165,28
6,5644231070190291328,124.603975,-30.666205,0.564147,0.014188,-2.911599,0.012019,2.692449,0.013831,11.260457,11.326833,11.042264,0.284569,0.882996,28
7,5644231035832878336,124.643384,-30.636666,0.569823,0.025049,-2.976160,0.021913,2.652526,0.023311,11.332324,11.401725,11.187494,0.214231,1.325736,28
8,5644231001470763264,124.643914,-30.643934,0.549875,0.033988,-3.297373,0.028696,2.811160,0.030932,11.365152,11.439619,11.222404,0.217216,2.042222,26
9,5644231070190288000,124.607305,-30.664758,0.564983,0.016472,-2.978209,0.013012,2.695124,0.016607,11.365314,11.509143,11.106326,0.402817,1.053840,27


In [11]:
df.shape

(929, 15)